<a href="https://colab.research.google.com/github/hmyrcmn/FaceMaskDetectionYolo/blob/main/jsonConverttoYolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json

def json_to_yolov8(json_path, label_mapping, image_width, image_height):
    with open(json_path, 'r') as file:
        json_data = json.load(file)

    image_path = json_data.get("imagePath", "")
    shapes = json_data.get("shapes", [])

    labels = []

    for shape in shapes:
        label = shape.get("label", "")
        points = shape.get("points", [])

        if label in label_mapping:
            class_id = label_mapping[label]
            for point in points:
                x, y = point
                # Görüntü boyutlarına göre normalize et
                normalized_x = x / image_width
                normalized_y = y / image_height
                labels.append(f"{class_id} {normalized_x} {normalized_y} 1.0 1.0")

    return image_path, labels

def process_json_files(json_folder, output_folder, image_width, image_height):
    label_mapping = {
        "maskesiz": 1,
        "maskeli": 2,
        "boneli": 3,
        "maskesiz": 4,
        "onluklu": 5,
        "onluksuz": 6
    }  # Burada kendi etiketlerinizi ve numaralandırmalarınızı ekleyin.

    for filename in os.listdir(json_folder):
        if filename.endswith(".json"):
            json_path = os.path.join(json_folder, filename)
            image_path, labels = json_to_yolov8(json_path, label_mapping, image_width, image_height)

            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")

            with open(output_path, 'w') as output_file:
                output_file.write('\n'.join(labels))

            print(f"Processed: {json_path} -> {output_path}")

# JSON dosyalarının bulunduğu klasör, çıkış klasörü, görüntü genişliği ve yüksekliğini belirtin
json_folder = "/content/data"
output_folder = "/content/labels"
image_width = 800  # Görüntü genişliği
image_height = 600  # Görüntü yüksekliği

# İşlemi başlat
process_json_files(json_folder, output_folder, image_width, image_height)


In [ ]:
import os
from zipfile import ZipFile

def zip_directory(directory_path, zip_path):
    with ZipFile(zip_path, 'w') as zipf:
        for root, dirs, files in os.walk(directory_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, directory_path)
                zipf.write(file_path, arcname=arcname)

# Kullanım örneği
directory_to_zip = '/content/labels'
zip_file_path = '/content/labels.zip'

zip_directory(directory_to_zip, zip_file_path)
